In [8]:
from os.path import isdir, join
import pickle
import numpy as np
import matplotlib.pyplot as plt
# import rosbag
import glob
from sklearn.model_selection import train_test_split, KFold
import os
import pdb
import math
import torch
from torch import nn

In [6]:
dataset_filepath = 'short_prediction_data.pt'
dataset = torch.load(dataset_filepath)


xobs_train, xpred_train, yintention_train, xobs_test, xpred_test, yintention_test = \
dataset["xobs_train"], dataset["xpred_train"], dataset["yintention_train"], dataset["xobs_test"], \
dataset["xpred_test"], dataset["yintention_test"]
obs_seq_len, pred_seq_len = dataset["obs_seq_len"], dataset["pred_seq_len"]


In [ ]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
for train_index, validation_index in kf.split(xobs_train):
    xobs_train_k, xobs_val_k = xobs_train[train_index], xobs_train[validation_index]
    xpred_train_k, xpred_val_k = xpred_train[train_index], xpred_train[validation_index] 
    yintention_train_k, yintention_val_k = yintention_train[train_index], yintention_train[validation_index]


In [ ]:
from torch.utils.data import Dataset
import os
import numpy as np
import math
import torch
from src.mgnn.utils import seq_to_graph

class TrajectoriesDataset(Dataset):
    def __init__(
        self,
        xobs,
        xpred,
        yintention,
        obs_seq_len,
        pred_seq_len,
        ):
        super(TrajectoriesDataset, self).__init__()
        self.obs_seq_len = obs_seq_len
        self.pred_seq_len = pred_seq_len
        self.seq_len = self.obs_seq_len + self.pred_seq_len
        
        


    def __len__(self):
        return self.num_seq


    def __getitem__(self, index):
        start, end = self.seq_start_end[index]
        out = [
            self.obs_traj[start:end, :], self.pred_traj[start:end, :],
            self.obs_traj_rel[start:end, :], self.pred_traj_rel[start:end, :],
            self.loss_mask_rel[start:end, :], self.loss_mask[start:end, :],
            self.v_obs[index], self.A_obs[index],
            self.v_pred[index], self.A_pred[index],
            self.attn_mask_obs[index], self.attn_mask_pred[index],
        ]
        return out

In [ ]:
import sys
pkg_path = '..'
sys.path.append(pkg_path)
import pickle
import torch
from torch import nn
from src_v2.utils import average_offset_error, padding_mask
import pdb


class IntentionLstm(nn.Module):

    def __init__(self, embedding_size=64, hidden_size=64, num_layers=1, \
                 dropout=0., device='cuda:0'):
        """num_lstms indicates the number of LSTMs stacked together."""
        super(IntentionLstm, self).__init__()
        self.embedding_size, self.hidden_size, self.num_layers = \
            embedding_size, hidden_size, num_layers

        self.lstm = nn.LSTM(
                    input_size=embedding_size,
                    hidden_size=hidden_size,
                    num_layers=num_layers,
                    batch_first=True,
                    dropout=dropout,
                    bidirectional=False,
                    ).to(device)

        self.spatial_embedding = nn.Linear(4, embedding_size)
        self.hidden2pos = nn.Linear(hidden_size, 2)
        
        
    
    def init_hidden(self, batch_size, device='cuda:0'):
        return (
            torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
            torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
        )
    
    def get_hf(self, so, sp, si, mode='teacher_forcing', device='cuda:0'):

        """
        Let's first pretend there is no corner case and everything is fine.
        inputs:
            - so # observation sample. # tensor. size: (N, obs_seq_len, 2) # already .to(device)
            - sp # ground truth prediction sample. # tensor. size: (N, pred_seq_len, 2) # already .to(device)
            - si # intention sample. # tensor. size: (N, 1, 2) # already .to(device)
            - mode # 'teacher_forcing' or 'no_forcing'.

        outputs:
            
        """
        batch_size = so.size(0) # batch_size = 
        sio = torch.ones_like(so).to(device) * si # attach to so
        so_ebd = torch.cat((so, sio), dim=2) # (1, obs_seq_len, 4)
        so_ebd = self.spatial_embedding(so_ebd.reshape(-1, 4))

        so_ebd = so_ebd.reshape(batch_size, -1, self.embedding_size)
        hc_0 = self.init_hidden(batch_size, device=device)
        out, hc_t = self.lstm(so_ebd, hc_0) #  out: (batch_size, obs_seq_len, hidden_size)
        hf = out[:, -1:, :] # (batch_size, 1, hidden_size)
    
        return hf
    
    def forward(self, so, so_lens, sp, sp_lens, si, mode='teacher_forcing', device='cuda:0'):
        """
        Parallel computation.
        inputs:
            - so # observation sample. # tensor. size: (N, T_obs_max, 2)
            - so_lens # T_obs for all samples in sp. # tensor. size: (N,)
            - sp # ground truth prediction sample. # tensor. size: (N, T_pred_max, 2)
            - sp_lens # T_pred for all samples in sp. # tensor. size: (N,)
            - si # intention sample. # tensor. size: (N, 1, 2)
            - mode # 'teacher_forcing' or 'no_forcing'.

        outputs:
            - loss # average offset error across samples.
            - sp_pred # prediction on sp. # tensor. size: (N, T_pred_max, 2)
        """
        so_intent = torch.cat((so, si * torch.ones_like(so)), dim=2).float().to(device) # N, T_obs_max, 4
        sp_intent = torch.cat((sp, si * torch.ones_like(sp)), dim=2).float().to(device) # N, T_pred_max, 4
        si = si.float().to(device)
        sp = sp.float().to(device)
        sp_mask = padding_mask(sp_lens).float().to(device) # (N, T_pred_max, 1)

        batch_size, _, input_channel = so_intent.shape
        T_pred_max = sp.shape[1]

        so_intent_ebd = self.spatial_embedding(so_intent.reshape(-1, input_channel))
        so_intent_ebd = so_intent_ebd.reshape(batch_size, -1, self.embedding_size) # (N, T_obs_max, embed_size)

        packed_so_intent_ebd = torch.nn.utils.rnn.pack_padded_sequence(so_intent_ebd, so_lens, batch_first=True, enforce_sorted=False)

        out, hc_t = self.lstm(packed_so_intent_ebd)

        out, _ = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)


        so_last_idx = (torch.ones(batch_size, 1, self.hidden_size)*(so_lens-1).reshape(-1,1,1)).long().to(device) # so_lens-1 -> len-1 is idx
        out = out.gather(1, so_last_idx) # dim=1 -> time dimension # (batch_size, 1, hidden_size)

        pred_list = []

        pred_ts = self.hidden2pos(out.reshape(batch_size, -1)).reshape(batch_size, 1, 2) # (batch, 1, 2)

        pred_list.append(pred_ts)
        pred_ts_past = pred_ts
        for ts in range(1, T_pred_max):
            if mode == 'teacher_forcing':
                pred_ts_past_intent = sp_intent[:, ts-1:ts] # torch.cat((pred_ts_past, si), dim=2) # (batch, 1, 4) 
            elif mode == 'no_forcing':
                pred_ts_past_intent = torch.cat((pred_ts_past, si), dim=2) # (batch, 1, 4)
            pred_ts_past_intent_ebd = self.spatial_embedding(pred_ts_past_intent.reshape(-1, 4))
            pred_ts_past_intent_ebd = pred_ts_past_intent_ebd.reshape(batch_size, 1, self.embedding_size)
            out, hc_t = self.lstm(pred_ts_past_intent_ebd, hc_t) #(N, 1, embed_size)
            pred_ts = self.hidden2pos(out.reshape(batch_size, -1)).reshape(batch_size, 1, 2)
            pred_list.append(pred_ts)
            pred_ts_past = pred_ts

        sp_pred = torch.cat(pred_list, dim=1) # (N, T_pred_max, 2)
        loss = average_offset_error(sp, sp_pred, sp_mask)
        return loss, sp_pred

    

    
    def forward_origin(self, so, sp, si, mode='teacher_forcing', device='cuda:0'):

        """
        Let's first pretend there is no corner case and everything is fine.
        inputs:
            - so # observation sample. # tensor. size: (N, obs_seq_len, 2) # already .to(device)
            - sp # ground truth prediction sample. # tensor. size: (N, pred_seq_len, 2) # already .to(device)
            - si # intention sample. # tensor. size: (N, 1, 2) # already .to(device)
            - mode # 'teacher_forcing' or 'no_forcing'.

        outputs:
            
        """
        batch_size = so.size(0) # batch_size = 
        sio = torch.ones_like(so).to(device) * si # attach to so
        so_ebd = torch.cat((so, sio), dim=2) # (1, obs_seq_len, 4)
        so_ebd = self.spatial_embedding(so_ebd.reshape(-1, 4))

        so_ebd = so_ebd.reshape(batch_size, -1, self.embedding_size)
        hc_0 = self.init_hidden(batch_size, device=device)
        out, hc_t = self.lstm(so_ebd, hc_0) #  out: (batch_size, obs_seq_len, hidden_size)
        hf = out[:, -1:, :] # (batch_size, 1, hidden_size)

        sp_pred = torch.zeros_like(sp)
        sp_pred_ts = self.hidden2pos(hf.reshape(batch_size, -1)).reshape(batch_size, 1, 2)
        sp_pred[:, 0:1, :] = sp_pred_ts
        sp_pred_ts_prev = sp_pred_ts

        pred_seq_len = sp.size(1)

        for ts in range(1, pred_seq_len):
            if mode == 'teacher_forcing':
                sp_ts_prev = sp[:, ts-1:ts] # (batch_size, 1, 2)
                sp_pred_ts_prev_ebd = torch.cat((sp_ts_prev, si), dim=2) # (1, 1, 4)
                sp_pred_ts_prev_ebd = self.spatial_embedding(sp_pred_ts_prev_ebd .reshape(-1, 4)) # embedding from ground truth data
            elif mode == 'no_forcing':
                sp_pred_ts_prev_ebd = torch.cat((sp_pred_ts_prev, si), dim=2) # (1, 1, 4)
                sp_pred_ts_prev_ebd = self.spatial_embedding(sp_pred_ts_prev_ebd.reshape(-1, 4))
            else:
                print('Error on mode.')
                sys.exit(1)

            sp_pred_ts_prev_ebd = sp_pred_ts_prev_ebd.reshape(batch_size, 1, self.embedding_size)

            out, hc_t = self.lstm(sp_pred_ts_prev_ebd, hc_t)
            hf = out[:, -1:, :]
            
            sp_pred_ts = self.hidden2pos(hf.reshape(batch_size, -1)).reshape(batch_size, 1, 2)
            sp_pred[:, ts:ts+1, :] = sp_pred_ts
            sp_pred_ts_prev = sp_pred_ts

#         loss = torch.mean(((sp_pred-sp) ** 2).sum(2))
        loss = torch.mean((((sp_pred-sp) ** 2).sum(2))**(1./2))

        return loss, sp_pred
